# Install detectron2
설치 설명: https://detectron2.readthedocs.io/en/latest/tutorials/install.html  </br>
코랩 내 설명 : https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=FsePPpwZSmqt

In [69]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [70]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [80]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Openpose 설치
https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/installation/0_index.md#windows-portable-demo

In [6]:
!git clone 'https://github.com/CMU-Perceptual-Computing-Lab/openpose'

Cloning into 'openpose'...
remote: Enumerating objects: 16111, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 16111 (delta 6), reused 13 (delta 2), pack-reused 16089
Receiving objects: 100% (16111/16111), 84.46 MiB | 40.91 MiB/s, done.
Resolving deltas: 100% (11284/11284), done.


In [12]:
!cd openpose/


import subprocess
# Submodule 초기화 및 업데이트 명령어 실행
subprocess.run(['git', 'submodule', 'update', '--init', '--recursive', '--remote'])


CompletedProcess(args=['git', 'submodule', 'update', '--init', '--recursive', '--remote'], returncode=128)

# Carvekit 설치
https://github.com/OPHoperHPO/freezed_carvekit_2023

In [34]:
# Check if CUDA is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")

CUDA available: False


In [35]:
# cpu버전 설치
!pip install git+https://github.com/OPHoperHPO/freezed_carvekit_2023 --extra-index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
  Cloning https://github.com/OPHoperHPO/freezed_carvekit_2023 to /tmp/pip-req-build-ujhbjol_
  Running command git clone --filter=blob:none --quiet https://github.com/OPHoperHPO/freezed_carvekit_2023 /tmp/pip-req-build-ujhbjol_
  Resolved https://github.com/OPHoperHPO/freezed_carvekit_2023 to commit e4acb6180d790c140701c570a9e59febf3bbc765
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8

# Densepose 설치
detectron2에 포함되어있음</br>
사용 설명 : https://github.com/facebookresearch/detectron2/blob/main/projects/DensePose/doc/GETTING_STARTED.md
</br>
예시 모델 다운로드 : https://github.com/facebookresearch/detectron2/blob/main/projects/DensePose/doc/DENSEPOSE_IUV.md#ModelZoo

</br>
예제에서 다운로드한 모델 : model_final_d366fa.pkl</br>


# PreProcessing code test

In [72]:
import numpy as np
from PIL import Image
from scipy.ndimage import sobel
import cv2
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import openpose as op
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import torch
from carvekit.api.high import HiInterface # 배경삭제


In [73]:
import numpy as np
from PIL import Image
from scipy.ndimage import sobel
import cv2
from torch.utils.data import Dataset
from torchvision import transforms
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import torch
from carvekit.api.high import HiInterface  # 배경삭제

class TryOnDiffusionDataset(Dataset):
    def __init__(self, person_image_paths, garment_image_paths, densepose_config_path, densepose_weights_path, transform=None):
        self.person_image_paths = person_image_paths
        self.garment_image_paths = garment_image_paths
        self.transform = transform
        self.densepose_predictor = self._init_densepose_predictor(densepose_config_path, densepose_weights_path)
        self.bg_remover = HiInterface(object_type="object")

    def __len__(self):
        return len(self.person_image_paths)

    def __getitem__(self, idx):
        # Load person image
        person_image_path = self.person_image_paths[idx]
        person_image = Image.open(person_image_path)

        # Predict human parsing map and 2D pose keypoints for person image using DensePose
        Sp = self.get_densepose_output(person_image)

        # Load garment image (assuming you have a separate list of garment image paths)
        garment_image_path = self.garment_image_paths[idx]
        garment_image = Image.open(garment_image_path)

        # Predict human parsing map and 2D pose keypoints for garment image using DensePose
        Jg = self.get_densepose_output(garment_image)

        # Segment out the garment from the garment image
        Ic = self.segment_clothing(garment_image, Sp)

        # Generate clothing-agnostic RGB image for the person image
        Ia = self.generate_clothing_agnostic_image(person_image, Sp)

        if self.transform:
            Ia = self.transform(Ia)
            Ic = self.transform(Ic)

        return Ia, Sp, Ic, Jg

    def _init_densepose_predictor(self, config_path, weights_path):
        cfg = get_cfg()
        cfg.merge_from_file(config_path)
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
        cfg.MODEL.WEIGHTS = weights_path
        return DefaultPredictor(cfg)

    def get_densepose_output(self, image):
        # 이미지를 OpenCV 형식으로 변환
        image_cv2 = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        # DensePose 예측 수행
        outputs = self.densepose_predictor(image_cv2)
        # 파싱 맵 추출
        densepose = outputs["instances"].pred_densepose
        # S는 파싱 맵을 나타내는 텐서입니다.
        parsing_map = densepose.S.cpu().numpy()[0]
        return parsing_map

    def segment_clothing(self, image, parsing_map):
        # 파싱 맵에서 의복 영역을 나타내는 라벨 값 (예: 2)을 사용하여 마스크 생성
        clothing_mask = (parsing_map == 2).astype(np.uint8)
        image_np = np.array(image)
        Ic = image_np * np.repeat(clothing_mask[:, :, np.newaxis], 3, axis=2)
        return Image.fromarray(Ic)

    def generate_clothing_agnostic_image(self, image, parsing_map):
        # 파싱 맵에서 의복을 제외한 나머지 영역을 사용하여 마스크 생성
        non_clothing_mask = (parsing_map != 2).astype(np.uint8)
        image_np = np.array(image)
        Ia = image_np * np.repeat(non_clothing_mask[:, :, np.newaxis], 3, axis=2)
        return Image.fromarray(Ia)


01. yaml 파일에 MODEL.DENSEPOSE_ON 내용 추가

In [84]:
person_image_paths = "/content/person_file/person.jpg"
garment_image_paths = "/content/cloth_file/cloth.jpg"
densepose_config_path = "/content/detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml"
densepose_weights_path = "/content/model_final_d366fa.pkl"
transform = None

# 데이터셋 클래스 생성
dataset = TryOnDiffusionDataset(person_image_paths, garment_image_paths, densepose_config_path, densepose_weights_path, transform)


WARNING [08/24 08:04:41 d2.config.compat]: Config '/content/detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


KeyError: ignored